In [2]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import pickle

dataset_dir = 'Dataset/Push_to_Talk_Corpus/RefExp'
success_dir = 'Dataset/Push_to_Talk_Corpus/Success'

In [3]:
move_level_data = {'alignment e':[], 'alignment p': [], 'coherence e':[], 'coherence p': []}

for dialogue in listdir(dataset_dir):
    #extract dialogue type
    success_file = dialogue.replace('refexps', 'movesuccess')
    success_file = path.join(success_dir, success_file)
    dialogue = path.join(dataset_dir, dialogue)
    #load referring expressions data
    dialogue = refexps.load_data(dialogue)
    success_moves = refexps.load_data(success_file)
    #move level
    dialogue_move = refexps.move_level(dialogue)
    #iterate over moves
    for move in dialogue_move:
        values = [[],[],[],[]]
        success = success_moves[move[0]]
        if success == 'no_video':
            continue
        refs = move[1]
        #iterate over objects
        for obj in refs:
            #iterate over objects
            refobj = refs[obj]
            sim_align = []
            sim_align_speaker = defaultdict(list)
            sim_coherence_speaker = defaultdict(list)
            sim_coherence = []
            for ref in refobj:
                speaker = ref[1]
                sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj))
                if sim != 'undefined':
                    sim_align.append(sim)
                    sim_align_speaker[speaker].append(sim)
            if sim_align == []:
                sim_align = np.NaN
            else:
                sim_align = np.mean(sim_align)
            comb_ref = combinations(refobj,2)
            for comb in comb_ref:
                if comb[0] != comb[1]:
                    sim = refexps.lex_sim(comb[0], comb[1])
                    if sim != 'undefined':
                        sim_coherence.append(sim)
                        sim_coherence_speaker[comb[0][1]].append(sim)
                        sim_coherence_speaker[comb[1][1]].append(sim)
            #movelevel coherence and alignment (for each object)
            if sim_coherence == []:
                sim_coherence = np.NaN
            else:
                sim_coherence = np.mean(sim_coherence)
            for s in sim_align_speaker: 
                if not np.isnan(np.mean(sim_align_speaker[s])):
                    move_level_data['alignment '+ s.replace('-utts', '')].append((np.mean(sim_align_speaker[s]), success))
            for s in sim_coherence_speaker:
                if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                    move_level_data[ 'coherence '+ s.replace('-utts', '')].append((np.mean(sim_coherence_speaker[s]), success))

pickle.dump(move_level_data, open('regression_lexical.p', 'w'))